In [11]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b

In [12]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [13]:
market = Market()
speculation_db = ADatabase("FS")

In [14]:
market.connect()
prices = market.retrieve("prices")
sp500 = market.retrieve("sp500")
market.disconnect()
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [15]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [16]:
simulation = prices.merge(simulation.drop(["adjclose","date"],axis=1),on=["year","quarter","ticker"],how="left")

In [17]:
simulation["delta"] = (simulation["prediction"] - simulation["adjclose"]) / simulation["adjclose"]

In [18]:
sim = simulation.dropna()

In [19]:
trades = []
reqs = [0.1,0.2,0.3]
values = [True,False]
parameters = []
positions = 10
for value in values:
    for signal in reqs:
        for req in reqs:
            parameter = {"value":value,"signal":signal,"req":req}
            parameters.append(parameter)

In [20]:
speculation_db.connect()
for parameter in tqdm(parameters):
    iteration_trades = b.financial_backtest(sim,parameter,start_date,end_date)
    speculation_db.store("trades",pd.DataFrame(iteration_trades))
speculation_db.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [03:50<00:00, 12.83s/it]
